# Prophet - Predecir el valor de Bitcoin


Prophet es una de las librerías más avanzadas para predecir series temporales desarrollada por Facebook. Te enseñaremos a como entrenar un modelo con Prophet, a optimizarlo y a utilizarlo para realizar predicciones futuras. En este ejercicio vamos a practicar a predecir el valor de Bitcoin, una criptomoneda. Es la criptomoneda que le ha marcado el camino a todas las demás que llegaron después utilizando su tecnología.  

<hr/>
<div class="alert alert-success alertsuccess" style="margin-top: 20px">
[Tip]: Para ejecutar el código de Python en la celda de código a continuación, haz clic en la celda para seleccionarla y presiona <kbd>Shift</kbd> + <kbd>Enter</kbd>.
</div>
<hr/>


In [2]:
#Dependencias de Prophet
!pip install pystan==2.19.1.1 --quiet
!pip install cmdstanpy --quiet
# Instalamos el modelo predictivo Prophet
!pip install fbprophet --quiet
# De aquí vamos a descargar los datos
!pip install yfinance --quiet

In [3]:
#Importamos las dependencias
import pandas as pd
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import plotly.graph_objects as go
from fbprophet import Prophet
from fbprophet.plot import plot_plotly, plot_components_plotly
import warnings

warnings.filterwarnings('ignore')

pd.options.display.float_format = '${:,.2f}'.format

<h3 id="version">Cargar histórico de datos</h3>


<p>
    Para descargarnos la serie temporal de BTC lo vamos a hacer desde Yahoo Finance. En concreto vamos a descargar el histórico desde 2016 hasta la actualidad.
</p>



In [4]:
# Fecha de inicio es un string 2016-01-01 y fecha de fin (hoy) en formato texto '%Y-%m-%d'
today = '2023-02-08'
start_date = '2016-01-01'
# Descargar el dataframe
eth_df = yf.download('ETH-EUR',start_date, today).reset_index()

eth_df.tail()

[*********************100%***********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
1910,2023-02-03,"$1,506.60","$1,542.41","$1,500.01","$1,540.64","$1,540.64",7560482103
1911,2023-02-04,"$1,540.39","$1,563.26","$1,525.32","$1,541.95","$1,541.95",5404762658
1912,2023-02-05,"$1,542.05","$1,544.15","$1,498.67","$1,512.28","$1,512.28",6419980973
1913,2023-02-06,"$1,512.28","$1,540.52","$1,502.30","$1,506.40","$1,506.40",6449569713
1914,2023-02-07,"$1,506.18","$1,560.21","$1,504.95","$1,558.16","$1,558.16",7483487813


In [5]:
# Validamos que no hay datos vacíos
eth_df.isnull().sum()

Date         0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [6]:
# TAREA: Filtramos el dataframe para quedarnos solo las columnas "Date" y "Open"
# Date será la fecha del valor y Open el valor del BTC en ese momento
df = eth_df[['Date', 'Open']]

# Valores que espera el prophet
# Eje X: Indíce llamado ds
# Eje Y: Serie a predecir llamada y
# TAREA HECHA: Crear un diccionario con clave "Date" y valor "ds"
# Y otra clave "Open" y valor "y"
new_names = {
    "Date": "ds", 
    "Open": "y"
}

# TAREA: Utiliza el diccionario para renombrar las columnas de df
df = df.rename(columns=new_names)

df

,ds,y
0,2017-11-11,$256.01
1,2017-11-12,$269.82
2,2017-11-13,$263.31
3,2017-11-14,$271.60
4,2017-11-15,$286.68
...,...,...
1910,2023-02-03,"$1,506.60"
1911,2023-02-04,"$1,540.39"
1912,2023-02-05,"$1,542.05"
1913,2023-02-06,"$1,512.28"


In [7]:
#Ejecuta la siguiente línea, si lo has hecho bien hasta aquí no dará error
df['ds'] = df['ds'].dt.tz_localize(None)

In [8]:
df.tail()

,ds,y
1910,2023-02-03,"$1,506.60"
1911,2023-02-04,"$1,540.39"
1912,2023-02-05,"$1,542.05"
1913,2023-02-06,"$1,512.28"
1914,2023-02-07,"$1,506.18"


In [20]:
# Crear la gráfica del precio de apertura
#Asigna a la variable x la columna ds
x = df["ds"]
#Asigna a la variable y la columna y
y = df["y"]

fig = go.Figure()

fig.add_trace(go.Scatter(x=x, y=y))

# Le ponemos el título a la gráfica
titulo = "ETH-USD"
fig.update_layout(
    title_text=titulo,
)

fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list(
                [
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1a", step="year", stepmode="backward"),
                    dict(step="all"),
                ]
            )
        ),
        rangeslider=dict(visible=True),
        type="date",
    )
)

<h3 id="version">Entrenar y predecir el modelo</h3>


<p>
    Ahora que has visto en la gráfica de que datos partimos. Inicializamos el Prophet y lo entramos pasandole el dataframe
</p>



In [10]:
#TAREA: Inicializa Prophet en la variable m con seasonality_mode en modo "multiplicative"
m = Prophet(seasonality_mode='multiplicative')

In [11]:
#TAREA: Entrena el modelo m con el dataframe de datos df
m.fit(df)

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


<p>
    Le decimos al modelo m los días en futuro que queremos predecir, en este caso el próximo año.
</p>



In [14]:
# TAREA: generar un dataframe con los días que tenemos y los 365 días siguientes a hoy para predecirlo
# PISTA: puedes usar la función del prophet make_future_dataframe()
future = m.make_future_dataframe(365)
future.tail()

,ds
2275,2024-02-03
2276,2024-02-04
2277,2024-02-05
2278,2024-02-06
2279,2024-02-07


In [15]:
# En el caso que hayas elegido un valor o fondo tradicional, es decir, que no opere los fines
# de semana, descomenta la siguiente línea:

# future = future[ future['ds'].dt.dayofweek < 5 ] # Nos elimina los fines de semana de 'ds' para no predecirlos
future

,ds
0,2017-11-11
1,2017-11-12
2,2017-11-13
3,2017-11-14
4,2017-11-15
...,...
2275,2024-02-03
2276,2024-02-04
2277,2024-02-05
2278,2024-02-06


<p>
    Al modelo entrenado le pasamos el dataframe a predecir.
</p>



In [16]:
#Utiliza el modelo m para predecir el dataframe future
forecast = m.predict(future)

In [17]:
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
2275,2024-02-03,$-943.49,"$-1,982.05",$125.67
2276,2024-02-04,$-956.92,"$-1,966.50",$62.39
2277,2024-02-05,$-969.48,"$-2,043.95",$131.36
2278,2024-02-06,$-979.73,"$-2,053.13",$95.75
2279,2024-02-07,$-990.52,"$-2,082.72",$75.78


<p>
    Gráfica con el modelo entrenado y los valores de un año en futuro predecidos.
</p>



In [18]:
next_day = (datetime.today() + timedelta(days=1)).strftime('%Y-%m-%d')
forecast[forecast['ds'] == next_day]['yhat'].item()
plot_plotly(m, forecast)

In [19]:
plot_components_plotly(m, forecast)